***import***

In [ ]:
import os
import sys
import time
import urllib.request
import datetime
import time
import json
import pandas as pd
import re

from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib
import requests

In [ ]:
client_id = 'client_id'
client_secret = 'client_secret'

**기본 코드**

네이버 API는 한번 호출에 100개를 반환한다.  
한번의 코드에 500개 이상을 반환하도록 구현(하지만 이 코드도 한번에 1000개 까지만 가능)  
참고자료: https://yenpa.tistory.com/3

In [ ]:
def getresult(client_id,client_secret,query,display=100,start=1,sort='sim'):
    encText = urllib.parse.quote(query)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + \
    "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    # HTTP 응답 상태코드 확인
    rescode = response.getcode()
    if(rescode==200):
        # HTTP 응답 바디 읽기
        response_body = response.read()
        response_str = response_body.decode('utf-8')
        # JSON 문자열을 Python 객체로 변환
        response_json = json.loads(response_str)
    else:
        print("Error Code:" + rescode)
    # 검색 결과 아이템 리스트 반환
    return response_json['items']

In [ ]:
def remove_tag(my_str):
    ## 태그를 지우는 함수
    p = re.compile('(<([^>]+)>)')
    return p.sub('', my_str)

def sub_html_special_char(my_str):
    ## 특수문자를 나타내는 &apos;, &quot를 실제 특수문자로 변환
    p1 = re.compile('&lt;')
    p2 = re.compile('&gt;')
    p3 = re.compile('&amp;')
    p4 = re.compile('&apos;')
    p5 = re.compile('&quot;')

    result = p1.sub('\<', my_str)
    result = p2.sub('\>', result)
    result = p3.sub('\&', result)
    result = p4.sub('\'', result)
    result = p5.sub('\"', result)
    return result

### 주제: 미국 - 네이버블로그

In [ ]:
query = '미국'
display = 100
start = 1000
sort = 'sim'

result_USA = []
for i in range(0,10):
    start = 1 + 100*i # (1) 검색 시작 위치 계산
    result_usa = getresult(client_id, client_secret, query, display, start, sort) # (2) 검색 결과를 가져오는 함수 
    result_USA.append(result_usa)

In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_USA = [item for sublist in result_USA for item in sublist]

In [ ]:
len(result_USA)

네이버 블로그 link가 있는 블로그만 검색하기  
(10. WebScraping_Practice_answer_배지환님 코드 참고)

In [ ]:
titles = []
links = []
descriptions = []

p = re.compile('https://blog.naver.com/.+')
for item in result_USA:
    if p.match(item['link']): ## <link>태그의 문자열이 blog.naver.com/으로 시작하는 결과만 추출
        # 제목에 있는 HTML 특수 문자를 제거하고 태그를 제거한 뒤 저장
        title = sub_html_special_char(remove_tag(item['title']))
        link = item['link']
        description = item['description']
        links.append(link)
        # 링크로부터 웹 페이지의 HTML 가져오기
        html = requests.get(link)
        bs_obj01 = BeautifulSoup(html.text, 'html.parser')
        src_url = "https://blog.naver.com/" + bs_obj01.iframe["src"] # (1)
        # src URL로부터 웹 페이지의 HTML 가져오기
        html = requests.get(src_url)
        bs_obj02 = BeautifulSoup(html.text, 'html.parser')
        title = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        title = title[0].text.replace('\n','')
        text = bs_obj02.find_all('div', {'class':'se-main-container'})
        text = text[0].text.replace('\n','')
        
        titles.append(title)
        descriptions.append(text)
        
        time.sleep(1)

#(1) p에 맞는 링크를 찾기 위한 정규 표현식 패턴 컴파일

#(2) 'title'에서 HTML 태그와 특수 문자를 제거한 결과를 저장

#(3) iframe 태그의 src URL 생성

In [ ]:
result_dict_USA = {'title': titles, 'link': links, 'description': descriptions}
df_USA = pd.DataFrame.from_dict(result_dict_USA)

**주제: 미국 - 데이터프레임 중복 제거**

In [ ]:
df_USA_1 = pd.read_csv('미국(블로그)1-988row.csv')
df_USA_2 = pd.read_csv('미국(블로그)2-987row.csv')
df_USA_3 = pd.read_csv('미국(블로그)3-989row.csv')
df_USA_4 = pd.read_csv('미국(블로그)4-984row.csv')

In [ ]:
merged_df_USA = pd.merge(df_USA_1, df_USA_2, on=['title', 'link', 'description'], how='outer')
df_USA_12 = merged_df_USA.drop_duplicates()

In [ ]:
merged_df_USA = pd.merge(df_USA_3, df_USA_4, on=['title', 'link', 'description'], how='outer')
df_USA_34 = merged_df_USA.drop_duplicates()

In [ ]:
merged_df_USA = pd.merge(df_USA_12, df_USA_34, on=['title', 'link', 'description'], how='outer')
df_USA = merged_df_USA.drop_duplicates()

### 주제: 여행 - 네이버블로그

In [ ]:
query = '여행'
display = 100
start = 1
sort = 'sim'

result_TRIP = []
for i in range(0,10):
    start = 1 + 100*i # (1) 검색 시작 위치 계산
    result_trip = getresult(client_id, client_secret, query, display, start, sort) # (2) 검색 결과를 가져오는 함수 
    result_TRIP.append(result_trip)


In [ ]:
result_TRIP = [item for sublist in result_TRIP for item in sublist]

In [ ]:
len(result_TRIP)

In [ ]:
titles = []
links = []
descriptions = []

p = re.compile('https://blog.naver.com/.+')
for item in result_TRIP:
    if p.match(item['link']): ## <link>태그의 문자열이 blog.naver.com/으로 시작하는 결과만 추출
        title = sub_html_special_char(remove_tag(item['title']))
        link = item['link']
        description = item['description']
        links.append(link)
        
        html = requests.get(link)
        bs_obj01 = BeautifulSoup(html.text, 'html.parser')
        src_url = "https://blog.naver.com/" + bs_obj01.iframe["src"]
        
        html = requests.get(src_url)
        bs_obj02 = BeautifulSoup(html.text, 'html.parser')

        title_elements = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        if title_elements:
            title = title_elements[0].text.replace('\n', '')
        else:
            title = "No title found"
        
        text_elements = bs_obj02.find_all('div', {'class':'se-main-container'})
        if text_elements:
            text = text_elements[0].text.replace('\n','')
        else: 
            text = "No text found"

        # title = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        # title = title[0].text.replace('\n','')
        # text = bs_obj02.find_all('div', {'class':'se-main-container'})
        # text = text[0].text.replace('\n','')
        
        titles.append(title)
        descriptions.append(text)
        
        time.sleep(1)

In [ ]:
result_dict_TRIP = {'title': titles, 'link': links, 'description': descriptions}
df_TRIP = pd.DataFrame.from_dict(result_dict_TRIP)

**주제: 여행 - 데이터프레임 중복 제거**

In [ ]:
df_TRIP_1 = pd.read_csv('여행(블로그)1-990row.csv')
df_TRIP_2 = pd.read_csv('여행(블로그)2-991row.csv')
df_TRIP_3 = pd.read_csv('여행(블로그)3-993row.csv')

In [ ]:
merged_df_TRIP = pd.merge(df_TRIP_1, df_TRIP_2, on=['title', 'link', 'description'], how='outer')
df_TRIP_12 = merged_df_TRIP.drop_duplicates()

In [ ]:
merged_df_TRIP = pd.merge(df_TRIP_12, df_TRIP_3, on=['title', 'link', 'description'], how='outer')
df_TRIP = merged_df_TRIP.drop_duplicates()

### 주제: 네이버 - 네이버블로그

In [ ]:
query = '네이버'
display = 100
start = 1000
sort = 'sim'

result_NAVER = []
for i in range(0,10):
    start = 1 + 100*i # (1) 검색 시작 위치 계산
    result_naver = getresult(client_id, client_secret, query, display, start, sort) # (2) 검색 결과를 가져오는 함수 
    result_NAVER.append(result_naver)


In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_NAVER = [item for sublist in result_NAVER for item in sublist]


In [ ]:
len(result_NAVER)

In [ ]:
titles = []
links = []
descriptions = []

p = re.compile('https://blog.naver.com/.+') # (1)
for item in result_NAVER:
    if p.match(item['link']): ## <link>태그의 문자열이 blog.naver.com/으로 시작하는 결과만 추출
        title = sub_html_special_char(remove_tag(item['title'])) # (2)
        link = item['link']
        description = item['description']
        links.append(link)
        
        html = requests.get(link)
        bs_obj01 = BeautifulSoup(html.text, 'html.parser')
        src_url = "https://blog.naver.com/" + bs_obj01.iframe["src"] # (3)
        
        # 블로그 내용 파싱을 위해 src_url을 생성하고 요청을 보냄
        html = requests.get(src_url)
        bs_obj02 = BeautifulSoup(html.text, 'html.parser')
    
        # 제목 파싱
        title_elements = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        if title_elements:
            title = title_elements[0].text.replace('\n', '')
        else:
            title = "No title found"
        
        # 본문 내용 파싱
        text_elements = bs_obj02.find_all('div', {'class':'se-main-container'})
        if text_elements:
            text = text_elements[0].text.replace('\n','')
        else: 
            text = "No text found"

        # title = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        # title = title[0].text.replace('\n','')
        # text = bs_obj02.find_all('div', {'class':'se-main-container'})
        # text = text[0].text.replace('\n','')
        
        titles.append(title)
        descriptions.append(text)
        
        time.sleep(1)

In [ ]:
result_dict_NAVER = {'title': titles, 'link': links, 'description': descriptions}
df_NAVER = pd.DataFrame.from_dict(result_dict_NAVER)

**주제: 네이버 - 데이터프레임 중복 제거**

In [ ]:
df_USA_1 = pd.read_csv('네이버(블로그)1-933row.csv')
df_USA_2 = pd.read_csv('네이버(블로그)2-932row.csv')
df_USA_3 = pd.read_csv('네이버(블로그)3-929row.csv')

In [ ]:
merged_df_NAVER = pd.merge(df_USA_1, df_USA_2, on=['title', 'link', 'description'], how='outer')
df_NAVER_12 = merged_df_NAVER.drop_duplicates()

In [ ]:
merged_df_NAVER = pd.merge(df_USA_12, df_USA_3, on=['title', 'link', 'description'], how='outer')
df_NAVER = merged_df_NAVER.drop_duplicates()

### 주제: 네이버지도 - 네이버블로그

In [ ]:
query = '네이버지도'
display = 100
start = 1000
sort = 'sim'

result_NAVER_MAP = []
for i in range(0,10):
    start = 1 + 100*i # (1) 검색 시작 위치 계산
    result_naver_MAP = getresult(client_id, client_secret, query, display, start, sort) # (2) 검색 결과를 가져오는 함수 
    result_NAVER_MAP.append(result_naver_MAP)


In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_NAVER_MAP = [item for sublist in result_NAVER_MAP for item in sublist]


In [ ]:
len(result_NAVER_MAP)

In [ ]:
titles = []
links = []
descriptions = []

p = re.compile('https://blog.naver.com/.+')
for item in result_NAVER_MAP:
    if p.match(item['link']): ## <link>태그의 문자열이 blog.naver.com/으로 시작하는 결과만 추출
        title = sub_html_special_char(remove_tag(item['title']))
        link = item['link']
        description = item['description']
        links.append(link)
        
        html = requests.get(link)
        bs_obj01 = BeautifulSoup(html.text, 'html.parser')
        src_url = "https://blog.naver.com/" + bs_obj01.iframe["src"]
        
        html = requests.get(src_url)
        bs_obj02 = BeautifulSoup(html.text, 'html.parser')

        title_elements = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        if title_elements:
            title = title_elements[0].text.replace('\n', '')
        else:
            title = "No title found"
        
        text_elements = bs_obj02.find_all('div', {'class':'se-main-container'})
        if text_elements:
            text = text_elements[0].text.replace('\n','')
        else: 
            text = "No text found"

        # title = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        # title = title[0].text.replace('\n','')
        # text = bs_obj02.find_all('div', {'class':'se-main-container'})
        # text = text[0].text.replace('\n','')
        
        titles.append(title)
        descriptions.append(text)
        
        time.sleep(1)

In [ ]:
result_dict_NAVER_MAP = {'title': titles, 'link': links, 'description': descriptions}
df_NAVER_MAP = pd.DataFrame.from_dict(result_dict_NAVER_MAP)

**주제: 네이버지도 - 데이터프레임 중복 제거**

In [ ]:
df_NAVER_MAP_1 = pd.read_csv('네이버지도(블로그)-918row.csv')
df_NAVER_MAP_2 = pd.read_csv('네이버지도(블로그)2-915row.csv')

In [ ]:
merged_df_NAVER_MAP = pd.merge(df_NAVER_MAP_1, df_NAVER_MAP_2, on=['title', 'link', 'description'], how='outer')
df_NAVER_MAP = merged_df_NAVER_MAP.drop_duplicates()

### 주제: 네이버쇼핑 - 네이버블로그

In [ ]:
query = '네이버쇼핑'
display = 100
start = 1000
sort = 'sim'

result_NAVER_SHOP = []
for i in range(0,10):
    start = 1 + 100*i # (1) 검색 시작 위치 계산
    result_naver_SHOP = getresult(client_id, client_secret, query, display, start, sort) # (2) 검색 결과를 가져오는 함수 호출
    result_NAVER_SHOP.append(result_naver_SHOP)


In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_NAVER_SHOP = [item for sublist in result_NAVER_SHOP for item in sublist]


In [ ]:
len(result_NAVER_SHOP)

In [ ]:
titles = []
links = []
descriptions = []

p = re.compile('https://blog.naver.com/.+')
for item in result_NAVER_SHOP:
    if p.match(item['link']): ## <link>태그의 문자열이 blog.naver.com/으로 시작하는 결과만 추출
        title = sub_html_special_char(remove_tag(item['title']))
        link = item['link']
        description = item['description']
        links.append(link)
        
        html = requests.get(link)
        bs_obj01 = BeautifulSoup(html.text, 'html.parser')
        src_url = "https://blog.naver.com/" + bs_obj01.iframe["src"]
        
        html = requests.get(src_url)
        bs_obj02 = BeautifulSoup(html.text, 'html.parser')

        title_elements = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        if title_elements:
            title = title_elements[0].text.replace('\n', '')
        else:
            title = "No title found"
        
        text_elements = bs_obj02.find_all('div', {'class':'se-main-container'})
        if text_elements:
            text = text_elements[0].text.replace('\n','')
        else: 
            text = "No text found"

        # title = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        # title = title[0].text.replace('\n','')
        # text = bs_obj02.find_all('div', {'class':'se-main-container'})
        # text = text[0].text.replace('\n','')
        
        titles.append(title)
        descriptions.append(text)
        
        time.sleep(1)

In [ ]:
result_dict_NAVER_SHOP = {'title': titles, 'link': links, 'description': descriptions}
df_NAVER_SHOP = pd.DataFrame.from_dict(result_dict_NAVER_SHOP)

**주제: 네이버쇼핑 - 데이터프레임 중복 제거**

In [ ]:
df_NAVER_SHOP_1 = pd.read_csv('네이버쇼핑1(블로그)-989row.csv')
df_NAVER_SHOP_2 = pd.read_csv('네이버쇼핑2(블로그)-991row.csv')

In [ ]:
merged_df_NAVER_SHOP = pd.merge(df_NAVER_SHOP_1, df_NAVER_SHOP_2, on=['title', 'link', 'description'], how='outer')
df_NAVER_SHOP = merged_df_NAVER_SHOP.drop_duplicates()

### 주제: 네이버블로그 - 네이버블로그

In [ ]:
query = '네이버블로그'
display = 100
start = 1000
sort = 'sim'

result_NAVER_BLOG = []
for i in range(0,10):
    start = 1 + 100*i # (1) 검색 시작 위치 계산
    result_naver_BLOG = getresult(client_id, client_secret, query, display, start, sort)# (2) 검색 결과를 가져오는 함수 호출
    result_NAVER_BLOG.append(result_naver_BLOG)


In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_NAVER_BLOG = [item for sublist in result_NAVER_BLOG for item in sublist]


In [ ]:
len(result_NAVER_BLOG)

In [ ]:
titles = []
links = []
descriptions = []

p = re.compile('https://blog.naver.com/.+')
for item in result_NAVER_BLOG:
    if p.match(item['link']): ## <link>태그의 문자열이 blog.naver.com/으로 시작하는 결과만 추출
        title = sub_html_special_char(remove_tag(item['title']))
        link = item['link']
        description = item['description']
        links.append(link)
        
        html = requests.get(link)
        bs_obj01 = BeautifulSoup(html.text, 'html.parser')
        src_url = "https://blog.naver.com/" + bs_obj01.iframe["src"]
        
        html = requests.get(src_url)
        bs_obj02 = BeautifulSoup(html.text, 'html.parser')

        title_elements = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        if title_elements:
            title = title_elements[0].text.replace('\n', '')
        else:
            title = "No title found"
        
        text_elements = bs_obj02.find_all('div', {'class':'se-main-container'})
        if text_elements:
            text = text_elements[0].text.replace('\n','')
        else: 
            text = "No text found"

        # title = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        # title = title[0].text.replace('\n','')
        # text = bs_obj02.find_all('div', {'class':'se-main-container'})
        # text = text[0].text.replace('\n','')
        
        titles.append(title)
        descriptions.append(text)
        
        time.sleep(1)

In [ ]:
result_dict_NAVER_BLOG = {'title': titles, 'link': links, 'description': descriptions}
df_NAVER_BLOG = pd.DataFrame.from_dict(result_dict_NAVER_BLOG)

**주제: 네이버블로그 - 데이터프레임 중복 제거**

In [ ]:
df_NAVER_BLOG_1 = pd.read_csv('.csv')
df_NAVER_BLOG_2 = pd.read_csv('.csv')
df_NAVER_BLOG_3 = pd.read_csv('.csv')

In [ ]:
merged_df_NAVER_BLOG = pd.merge(df_NAVER_BLOG_1234, df_NAVER_BLOG_5, on=['title', 'link', 'description'], how='outer')
df_NAVER_BLOG = merged_df_NAVER_BLOG.drop_duplicates()

### 주제: 네이버클라우드 - 네이버뉴스

In [ ]:
query = '네이버클라우드'
display = 100
start = 1000
sort = 'sim'

result_NAVER_CLOUD = []
for i in range(0,10):
    start = 1 + 100*i # (1) 검색 시작 위치 계산
    result_naver_CLOUD = getresult(client_id, client_secret, query, display, start, sort) # (2) 검색 결과를 가져오는 함수 호출
    result_NAVER_CLOUD.append(result_naver_CLOUD)


In [ ]:
# 리스트 컴프리헨션을 사용하여 이중 리스트 풀기
result_NAVER_CLOUD = [item for sublist in result_NAVER_CLOUD for item in sublist]


In [ ]:
len(result_NAVER_CLOUD)

In [ ]:
titles = []
links = []
descriptions = []

p = re.compile('https://blog.naver.com/.+')
for item in result_NAVER_CLOUD:
    if p.match(item['link']): ## <link>태그의 문자열이 blog.naver.com/으로 시작하는 결과만 추출
        title = sub_html_special_char(remove_tag(item['title']))
        link = item['link']
        description = item['description']
        links.append(link)
        
        html = requests.get(link)
        bs_obj01 = BeautifulSoup(html.text, 'html.parser')
        src_url = "https://blog.naver.com/" + bs_obj01.iframe["src"]
        
        html = requests.get(src_url)
        bs_obj02 = BeautifulSoup(html.text, 'html.parser')

        title_elements = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        if title_elements:
            title = title_elements[0].text.replace('\n', '')
        else:
            title = "No title found"
        
        text_elements = bs_obj02.find_all('div', {'class':'se-main-container'})
        if text_elements:
            text = text_elements[0].text.replace('\n','')
        else: 
            text = "No text found"

        # title = bs_obj02.find_all('div', {'class': 'se-module se-module-text se-title-text'})
        # title = title[0].text.replace('\n','')
        # text = bs_obj02.find_all('div', {'class':'se-main-container'})
        # text = text[0].text.replace('\n','')
        
        titles.append(title)
        descriptions.append(text)
        
        time.sleep(1)

In [ ]:
result_dict_NAVER_CLOUD = {'title': titles, 'link': links, 'description': descriptions}
df_NAVER_CLOUD = pd.DataFrame.from_dict(result_dict_NAVER_CLOUD)